In [1]:
import os
import numpy as np

In [2]:
pred_root = '/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds'
os.path.isdir(pred_root)

True

In [3]:
blend_fname = 'OUTPUT--0.198x_P02_+_0.108x_P03_+_0.036x_P14_+_0.657x_P17--audio_mWA4.npy'
blend_fpath = os.path.join(pred_root, blend_fname)
os.path.isfile(blend_fpath)

True

In [7]:
def get_weight_dict(fname: str) -> dict:
    import re
    pattern = r'(\d+\.\d+)x_(P\d+)'
    return {prim: float(wt) for wt, prim in re.findall(pattern, fname)}

blend_weights = get_weight_dict(blend_fname)
blend_weights

{'P02': 0.198, 'P03': 0.108, 'P14': 0.036, 'P17': 0.657}

In [11]:
primitive_fnames = [ f'OUTPUT--1.000x_{k}' for k in blend_weights ]
for pi in range(len(primitive_fnames)):
    for pred_fname in os.listdir(pred_root):
        if primitive_fnames[pi] in pred_fname:
            break
    primitive_fnames[pi] = pred_fname
primitive_fpaths = [ os.path.join(pred_root, fname) for fname in primitive_fnames ]
list(zip(primitive_fpaths, [os.path.isfile(path) for path in primitive_fpaths]))

[('/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds/OUTPUT--1.000x_P02--audio_mWA2.npy',
  True),
 ('/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds/OUTPUT--1.000x_P03--audio_mWA5.npy',
  True),
 ('/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds/OUTPUT--1.000x_P14--audio_mWA4.npy',
  True),
 ('/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds/OUTPUT--1.000x_P17--audio_mLH0.npy',
  True)]

In [12]:
blend_vec = np.zeros(20)
for key in blend_weights:
    idx = int(key[1:])
    blend_vec[idx] = blend_weights[key]

blend_weights, blend_vec

({'P02': 0.198, 'P03': 0.108, 'P14': 0.036, 'P17': 0.657},
 array([0.   , 0.   , 0.198, 0.108, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.036, 0.   , 0.   , 0.657,
        0.   , 0.   ]))

In [13]:
blend_vec_nz = blend_vec[blend_vec.nonzero()]
blend_vec.nonzero(), blend_vec_nz

((array([ 2,  3, 14, 17]),), array([0.198, 0.108, 0.036, 0.657]))

In [14]:
primitives = []
for ppath in primitive_fpaths:
    primitives.append(np.load(ppath))

blend = np.load(blend_fpath)
blend.shape, blend, list(zip([primitive.shape for primitive in primitives], primitives))

((60, 147),
 array([[-1.4377362e-01,  1.9802904e+00, -1.7480753e-01, ...,
         -1.9643907e-03,  9.8629731e-01,  8.3882334e-03],
        [-1.4612135e-01,  1.9763017e+00, -1.7529739e-01, ...,
         -2.6231203e-03,  9.8600399e-01,  7.1580419e-03],
        [-1.4485551e-01,  1.9712105e+00, -1.7530197e-01, ...,
         -2.4743928e-03,  9.8415035e-01,  7.7062333e-03],
        ...,
        [-2.2614880e-01,  1.9227753e+00, -2.0500295e-02, ...,
         -4.1961772e-03,  1.0002228e+00,  7.1376869e-03],
        [-2.2990106e-01,  1.9213305e+00, -1.1138514e-02, ...,
         -6.2945662e-03,  9.9821579e-01,  6.5597231e-03],
        [-2.2909024e-01,  1.9226239e+00, -7.9893544e-03, ...,
         -4.4651879e-03,  1.0008119e+00,  7.5792866e-03]], dtype=float32),
 [((60, 147),
   array([[-0.2207267 ,  2.0915682 , -0.40578103, ..., -0.01219465,
            0.9753186 ,  0.00372095],
          [-0.2243205 ,  2.08701   , -0.4012279 , ..., -0.00739323,
            0.9757288 ,  0.00248346],
          [-

In [15]:
op_blend = np.sum([weight * primitive for weight, primitive in zip(blend_vec_nz, primitives)], axis=0)
op_blend.shape

(60, 147)

In [16]:
np.abs(op_blend - blend), np.mean(np.abs(blend)), (100 * np.mean(np.abs(op_blend - blend)) / np.mean(np.abs(blend)))

(array([[0.00025678, 0.00573552, 0.00132962, ..., 0.00465165, 0.00219363,
         0.0023146 ],
        [0.00140166, 0.00542164, 0.00417402, ..., 0.00474991, 0.00237513,
         0.00251926],
        [0.0016672 , 0.00607419, 0.00499976, ..., 0.00466907, 0.00132102,
         0.00388719],
        ...,
        [0.02270569, 0.00060248, 0.01010233, ..., 0.00432325, 0.00384861,
         0.00664938],
        [0.02312446, 0.00065696, 0.01127899, ..., 0.00478368, 0.00247097,
         0.00751852],
        [0.0243196 , 0.00135398, 0.01329175, ..., 0.00469287, 0.00144553,
         0.00785631]], dtype=float32),
 0.38926324,
 1.9660128517329585)

In [17]:
op_fpath = blend_fpath.replace('OUTPUT', 'OPBLEND')
op_fpath

'/home/aravind/work/hays-8903/mint/_expts/audio_with_primitives_aligned/preds/OPBLEND--0.198x_P02_+_0.108x_P03_+_0.036x_P14_+_0.657x_P17--audio_mWA4.npy'

In [18]:
np.save(op_fpath, op_blend)

*Looping this over all blended predictions*

In [20]:
num_primitives = 20

primitive_fnames = [ f'OUTPUT--1.000x_P{p:02d}' for p in range(num_primitives) ]
primitives = [None] * num_primitives
for pi in range(num_primitives):
    for pred_fname in os.listdir(pred_root):
        if primitive_fnames[pi] in pred_fname:
            break
    primitive_fnames[pi] = pred_fname
    primitives[pi] = np.load(os.path.join(pred_root, pred_fname))
list(zip(primitive_fnames, primitives))

[('OUTPUT--1.000x_P00--audio_mWA4.npy',
  array([[-3.78265493e-02,  1.98155594e+00,  4.95034039e-01, ...,
          -1.09768575e-02,  9.96747792e-01, -3.00687086e-03],
         [-3.31806913e-02,  1.98287868e+00,  5.03155291e-01, ...,
          -9.38957836e-03,  9.96590912e-01, -2.89549772e-03],
         [-3.21088880e-02,  1.98448586e+00,  5.05898833e-01, ...,
          -1.16482405e-02,  9.94670033e-01, -4.52359207e-04],
         ...,
         [ 3.00065074e-02,  1.96206212e+00,  4.69561934e-01, ...,
           3.51912994e-03,  9.82540607e-01,  1.09917829e-02],
         [ 4.18227799e-02,  1.96659112e+00,  4.69052881e-01, ...,
           3.32622323e-03,  9.85941052e-01,  7.49675278e-03],
         [ 4.65496480e-02,  1.96808338e+00,  4.64064360e-01, ...,
           2.91813631e-03,  9.91921127e-01,  6.01001922e-03]], dtype=float32)),
 ('OUTPUT--1.000x_P01--audio_mWA2.npy',
  array([[-3.7521079e-01,  1.9780550e+00, -3.4195888e-01, ...,
          -8.7187020e-03,  9.9374151e-01, -3.0421009e-03]

In [23]:
import glob, tqdm

for blend_fpath in tqdm.tqdm(glob.glob(os.path.join(pred_root, 'OUTPUT--0*.npy'))):
    blend_fname = os.path.basename(blend_fpath)
    blend_weights = get_weight_dict(blend_fname)

    blend_vec = np.zeros(num_primitives)
    for key in blend_weights:
        idx = int(key[1:])
        blend_vec[idx] = blend_weights[key]

    blend = np.load(blend_fpath)
    op_blend = np.sum([weight * primitive for weight, primitive in zip(blend_vec, primitives)], axis=0)
    op_fpath = blend_fpath.replace('OUTPUT', 'OPBLEND')
    np.save(op_fpath, op_blend)

    perc_diff = 100 * np.mean(np.abs(op_blend - blend)) / np.mean(np.abs(blend))
    print(f"file: {blend_fname} \t OPBLEND vs OUTPUT perc diff: {perc_diff}%")

100%|██████████| 80/80 [00:00<00:00, 511.83it/s]

file: OUTPUT--0.015x_P01_+_0.809x_P05_+_0.176x_P11--audio_mWA5.npy 	 OPBLEND vs OUTPUT perc diff: 1.3283985203191928%
file: OUTPUT--0.761x_P03_+_0.002x_P04_+_0.121x_P05_+_0.047x_P06_+_0.069x_P11--audio_mWA5.npy 	 OPBLEND vs OUTPUT perc diff: 1.6662237547754626%
file: OUTPUT--0.992x_P14_+_0.008x_P16--audio_mLH2.npy 	 OPBLEND vs OUTPUT perc diff: 0.43301209338137653%
file: OUTPUT--0.199x_P06_+_0.238x_P10_+_0.563x_P14--audio_mWA4.npy 	 OPBLEND vs OUTPUT perc diff: 1.6467372485377043%
file: OUTPUT--0.759x_P16_+_0.241x_P18--audio_mLH2.npy 	 OPBLEND vs OUTPUT perc diff: 1.2397615160483229%
file: OUTPUT--0.243x_P04_+_0.049x_P17_+_0.709x_P19--audio_mLH0.npy 	 OPBLEND vs OUTPUT perc diff: 1.7701453933721307%
file: OUTPUT--0.293x_P04_+_0.707x_P17--audio_mLH0.npy 	 OPBLEND vs OUTPUT perc diff: 1.762276411550258%
file: OUTPUT--0.000x_P00_+_0.086x_P08_+_0.095x_P09_+_0.673x_P12_+_0.146x_P17--audio_mWA4.npy 	 OPBLEND vs OUTPUT perc diff: 1.9612988969366176%
file: OUTPUT--0.096x_P01_+_0.009x_P14_+_0.6